In [ ]:
import pandas as pd
import numpy as np
import os

def load_data(datapath):
    csv_path = os.path.abspath(datapath)
    return pd.read_csv(csv_path, header=None)

def data_split(dataset):
    train = dataset.sample(frac=0.8)
    val = dataset.drop(train.index)
    train_y = train.iloc[:,0]
    train_x = train.drop(0, axis=1)
    val_y = val.iloc[:,0]
    val_x = val.drop(0, axis=1)
    return train_x, train_y, val_x, val_y

def main(datapath):
    train_x, train_y, val_x, val_y = data_split(load_data(datapath))
    pass


class MLP:

    def __init__(self, n_hidden, n_input_feat, n_classes, learning_rate):
        self.n_hidden = n_hidden #4
        self.n_input = n_input_feat #14
        self.n_output = n_classes #3
        self.lr = learning_rate #0.0001

        self.v = np.random.uniform(size=((self.n_input+1)*self.n_hidden,1))
        self.w = np.random.uniform(size=((self.n_hidden+1)*self.n_output,1))


    def hidden_layer(self, x, v):
        #x = vector of length 15
        #v = vector of length 60
        a = np.zeros(size=(self.n_hidden,1))
        #a = 0 vector of length 4
        for node in range(self.n_hidden): #0,1,2,3
            a_x = np.zeros(size=(self.n_input+1,1)) 
            #a_x = 0 vector of length 15
            for index in range(len(v)): #0,1,2,3,...,59,60
                if index % self.n_hidden == node:
                    a_x[index//self.n_hidden] = v[index]
            a[node] = np.dot(a_x,x)
        for entry in range(a):
            a[entry] = (lambda z: z*(z > 0))(entry)
        return a
    
    def output_layer(self, a, w):
        # modify a to add bias term
        #a = vector of length 5
        #w = vector of length 15
        y = np.zeros(size=(self.n_output,1))
        #y = 0 vector of length 3
        for node in range(self.n_output): #0,1,2
            y_a = np.zeros(size=(self.n_hidden+1,1)) 
            #y_a = 0 vector of length 5
            for index in range(len(w)): #0,1,2,3,...,14,15
                if index % self.n_output == node:
                    y_a[index//self.n_output] = w[index]
            y[node] = np.dot(y_a,a)
        for entry in range(y):
            y[entry] = (lambda z: z*(z > 0))(entry) #CHANGE TO SOME SORT OF SOFTMAX LAMBDA WITH ONE HOT ENCODING OKAY LOWER YOUR VOICE SIR
        return y

    def predict(self, input):
        x = [] #make x vector out of input and bias
        a = self.hidden_layer(x, self.v)
        y = self.output_layer(a, self.w)




main('wine.data')

In [22]:
import numpy as np
print(np.random.uniform(size=(5,1)))

[[0.06924711]
 [0.49533038]
 [0.54240531]
 [0.82560534]
 [0.71195889]]
